In [1]:
def getDatabaseConnection():
    import os
    import yaml
    from sqlalchemy import create_engine
    db_conn = os.path.join(os.getcwd(), "db_connections/db_connections.yml")
    with open(db_conn, 'r') as ymlfile:
        db_conns = yaml.load(ymlfile)
    redshift = db_conns['Redshift']
    # megavolt = db_conns['Megavolt']
    engine = 'postgresql://{username}:{password}@{host}:{port}/{database}'.format(username=redshift['username'],
                                                                                  password=redshift['password'],
                                                                                  host=redshift['host'],
                                                                                  port=redshift['port'],
                                                                                  database=redshift['database'])
    conn = create_engine(engine, connect_args={'sslmode': 'require'})
    return conn

In [2]:
def queryDatabase(conn):
    df = pd.read_sql_query("select * from ee.david_invoices_weather_matched_4", con=conn)
    return df

In [3]:
import pandas as pd
df = queryDatabase(getDatabaseConnection())

In [4]:
#df['invoicefromdt'] = pd.to_datetime(df['invoicefromdt_timestamp'])
#df['invoicetodt'] = pd.to_datetime(df['invoicetodt_timestamp'])

In [5]:
slice = df[df['perc_missing'] >= 0.5].copy()
slice_2 = df[df['perc_missing'] <= 0.5].copy()
slice_3 = df[df['perc_missing'] <= 0.1].copy()

In [6]:
len(df), len(slice_2)/len(df), len(slice_3)/len(df)

(1896045, 0.8697425430303606, 0.8219398801188791)

In [7]:
slice_3.reset_index(inplace=True)

In [8]:
slice_3.drop('index', axis=1, inplace=True)

In [9]:
slice_4 = slice_3[slice_3['avg_kwh_day_over_invoice'] <= 100].copy()
#slice_5 = slice_4[slice_4['avg_kwh_day_over_invoice'] >= 8].copy()

In [10]:
import numpy as np
np.isnan(slice_4['tavg_interval_avg']).any()

False

In [11]:
len(slice_4)

1542090

In [12]:
slice_4.columns

Index(['iethicalid', 'zip', 'invoicefromdt_timestamp', 'invoicetodt_timestamp',
       'duration', 'kwh', 'wban', 'dist', 'tavg_interval_sum',
       'tavg_interval_avg', 'num_missing', 'tavg_interval_sum_corrected',
       'perc_missing', 'avg_kwh_day_over_invoice', 'todt_month', 'todt_day',
       'invoice_num', 'kwh_lag', 'kwh_lag_2', 'daily_kwh_lag',
       'daily_kwh_lag_2', 'kwh_updating_avg', 'daily_kwh_updating_avg',
       'first_date', 'days_passed', 'month_avg_preceding',
       'month_avg_preceding_flag'],
      dtype='object')

In [13]:
slice_4.head()

,iethicalid,zip,invoicefromdt_timestamp,invoicetodt_timestamp,duration,kwh,wban,dist,tavg_interval_sum,tavg_interval_avg,...,kwh_lag,kwh_lag_2,daily_kwh_lag,daily_kwh_lag_2,kwh_updating_avg,daily_kwh_updating_avg,first_date,days_passed,month_avg_preceding,month_avg_preceding_flag
0,3,15212,2013-07-25,2013-08-23,30,384.0,14762,14.855786,2066.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2013-07-25,29,NaN,0
1,3,15212,2013-08-24,2013-09-24,32,408.0,14762,14.855786,2140.0,66.0,...,384.0,NaN,12.80,NaN,384.0,12.80,2013-07-25,61,NaN,0
2,3,15212,2013-09-25,2013-10-23,29,348.0,14762,14.855786,1734.0,59.0,...,408.0,384.0,12.75,12.80,396.0,12.77,2013-07-25,90,NaN,0
3,3,15212,2013-10-24,2013-11-22,30,372.0,14762,14.855786,1358.0,45.0,...,348.0,408.0,12.00,12.75,380.0,12.51,2013-07-25,120,NaN,0
4,3,15212,2013-11-23,2013-12-24,32,552.0,14762,14.855786,1074.0,33.0,...,372.0,348.0,12.40,12.00,378.0,12.48,2013-07-25,152,NaN,0


In [14]:
slice_5 = slice_4.sample(frac=1).reset_index(drop=True)
slice_5 = slice_5[np.isfinite(slice_5['daily_kwh_lag'])]
#slice_5

In [15]:
slice_5.head()

,iethicalid,zip,invoicefromdt_timestamp,invoicetodt_timestamp,duration,kwh,wban,dist,tavg_interval_sum,tavg_interval_avg,...,kwh_lag,kwh_lag_2,daily_kwh_lag,daily_kwh_lag_2,kwh_updating_avg,daily_kwh_updating_avg,first_date,days_passed,month_avg_preceding,month_avg_preceding_flag
0,106356,16335,2015-06-03,2015-07-01,29,212.0,4843,5.354224,1947.0,67.0,...,275.0,277.0,8.33,9.55,308.000000,10.07,2014-10-01,273,NaN,0
1,132917,19015,2015-05-26,2015-06-27,33,819.0,13739,13.397960,2503.0,75.0,...,589.0,464.0,19.63,14.50,482.400000,15.78,2014-12-29,180,NaN,0
2,138333,19512,2016-03-23,2016-04-19,28,294.0,54782,14.020544,1343.0,47.0,...,357.0,432.0,11.15,14.89,549.066667,18.65,2015-01-01,474,12.28,1
3,126381,19382,2015-07-02,2015-08-03,33,393.0,3761,19.094723,2414.0,73.0,...,340.0,296.0,11.33,9.54,416.571429,13.44,2014-12-04,242,NaN,0
4,114661,10031,2016-11-14,2016-12-15,32,660.0,94728,5.362459,1404.0,43.0,...,607.0,605.0,18.96,20.16,599.111111,18.95,2014-11-13,763,19.39,1


In [36]:
#train_columns = ['dist', 'tavg_interval_avg', 'todt_month', 'daily_kwh_lag', 'daily_kwh_lag_2', 'daily_kwh_updating_avg', 'avg']
train_columns = ['dist', 'tavg_interval_avg', 'todt_month', 'daily_kwh_lag', 'daily_kwh_updating_avg', 'month_avg_preceding_flag', 'month_avg_preceding', 'invoicenum']
target_col = ['avg_kwh_day_over_invoice']
filter_columns = ['dist', 'tavg_interval_avg', 'todt_month', 'daily_kwh_lag', 'daily_kwh_updating_avg', 'month_avg_preceding_flag', 'month_avg_preceding', 'avg_kwh_day_over_invoice', 'invoicenum']

In [37]:
slice_6 = slice_5.filter(filter_columns)

In [38]:
slice_6.head()

,dist,tavg_interval_avg,todt_month,daily_kwh_lag,daily_kwh_updating_avg,month_avg_preceding_flag,month_avg_preceding,avg_kwh_day_over_invoice
0,5.354224,67.0,7,8.33,10.07,0,NaN,7.31
1,13.397960,75.0,6,19.63,15.78,0,NaN,24.81
2,14.020544,47.0,4,11.15,18.65,1,12.28,10.50
3,19.094723,73.0,8,11.33,13.44,0,NaN,11.90
4,5.362459,43.0,12,18.96,18.95,1,19.39,20.62


In [39]:
#slice_6.drop('invoicefromdt_timestamp', axis=1, inplace=True)
#slice_6.drop('invoicetodt_timestamp', axis=1, inplace=True)
#slice_6.drop('first_date', axis=1, inplace=True)

In [40]:
slice_6.fillna(0, inplace=True)
slice_6.head()

,dist,tavg_interval_avg,todt_month,daily_kwh_lag,daily_kwh_updating_avg,month_avg_preceding_flag,month_avg_preceding,avg_kwh_day_over_invoice
0,5.354224,67.0,7,8.33,10.07,0,0.00,7.31
1,13.397960,75.0,6,19.63,15.78,0,0.00,24.81
2,14.020544,47.0,4,11.15,18.65,1,12.28,10.50
3,19.094723,73.0,8,11.33,13.44,0,0.00,11.90
4,5.362459,43.0,12,18.96,18.95,1,19.39,20.62


In [41]:
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.preprocessing import Imputer

In [42]:
#imp = Imputer(missing_values='NaN', strategy='mean', axis=1)
#imp.fit(slice_5['month_avg_preceding'])
#new = imp.fit_transform(slice_5)

In [43]:
train = slice_6[0:int(len(slice_6)*.666)].copy()
test = slice_6[int(len(slice_6)*.666):].copy()

In [44]:
train_X = train.as_matrix(columns=train_columns)
train_y = train.as_matrix(columns=target_col)
test_X = test.as_matrix(columns=train_columns)
test_y = test.as_matrix(columns=target_col)

In [45]:
from sklearn.ensemble import RandomForestRegressor
algo = RandomForestRegressor(n_estimators=150, min_samples_split=2)

In [46]:
algo.fit(train_X, train_y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
print('done')

In [27]:
preds = algo.predict(test_X)
preds = preds.reshape(len(preds), 1)

In [28]:
err_list = test_y - preds

In [31]:
import math

In [32]:
err_list_abs = list(map(lambda x: math.fabs(x), err_list))

In [33]:
sum(err_list_abs)/len(preds)

3.9412497960256583

In [34]:
(sum(err_list_abs)/len(preds))/np.average(test_y)

0.20447977856278779

In [ ]:
param_grid = {"n_estimators" : [150, 200]}

In [ ]:
algo_2 = RandomForestRegressor(random_state=30)

In [ ]:
grid_search = GridSearchCV(algo_2, param_grid, n_jobs=1, cv=5)

In [ ]:
grid_search.fit(train_X, train_y)

In [ ]:
print(grid_search.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
import math

In [ ]:
algo_n150 = RandomForestRegressor(n_estimators=150, min_samples_split=2)
algo_n150.fit(train_X, train_y)
preds = algo_n150.predict(test_X)

In [ ]:
preds = preds.reshape(len(preds), 1)

In [ ]:
err_list = test_y - preds

In [ ]:
err_list_abs = list(map(lambda x: math.fabs(x), err_list))
print(sum(err_list_abs)/len(preds))

In [ ]:
algo_n200 = RandomForestRegressor(n_estimators=200, min_samples_split=2)
algo_n200.fit(train_X, train_y)
preds = algo_n200.predict(test_X)
preds = preds.reshape(len(preds), 1)
err_list = test_y - preds
err_list_abs = list(map(lambda x: math.fabs(x), err_list))
print(sum(err_list_abs)/len(preds))

In [ ]:
test['predicted_daily_kwh'] = preds

In [ ]:
test[['avg_kwh_day_over_invoice', 'predicted_daily_kwh']]

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(test['invoicetodt'], test['avg_kwh_day_over_invoice'], "o", color='blue', alpha = 0.1)
plt.plot(test['invoicetodt'], test['predicted_daily_kwh'], "o", color='pink', alpha = 0.1)
plt.legend(loc='best')
plt.show()

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(test.index, test['avg_kwh_day_over_invoice'], "o", color='blue', alpha = 0.1)
plt.plot(test.index, test['predicted_daily_kwh'], "o", color='pink', alpha = 0.1)
plt.legend(loc='best')
plt.show()

In [ ]:
test['avg_kwh_day_over_invoice'][test.index[0]]

In [ ]:
import plotly.plotly as py
from plotly.graph_objs import *

In [ ]:
alpha = plt.figure()
plt.plot(test['invoicetodt'], test['avg_kwh_day_over_invoice'], "o", color='blue', alpha = 0.1)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
trace = go.Scatter(
    x = test['invoicetodt'],
    y = test['avg_kwh_day_over_invoice'],
    mode = 'markers'
)

In [ ]:
data = [trace]

In [ ]:
import plotly

In [ ]:
plotly.offline.iplot(data, filename='basic-scatter')

In [ ]:
pred_frame = test[['avg_kwh_day_over_invoice', 'predicted_daily_kwh']]

In [ ]:
pred_frame.head()

In [ ]:
pred_frame['abs_error'] = err_list_abs

In [ ]:
err_list_sq = list(map(lambda x: x*x, err_list_abs))

In [ ]:
pred_frame['sq_error'] = err_list_sq

In [ ]:
math.sqrt(np.average(pred_frame['sq_error']))

In [ ]:
trace = go.Scatter(
    x = pred_frame['avg_kwh_day_over_invoice'],
    y = pred_frame['abs_error'],
    mode = 'markers'
)

In [ ]:
data = [trace]

In [ ]:
#plotly.offline.iplot(data, filename='basic-scatter')

In [ ]:
x = pred_frame['avg_kwh_day_over_invoice']
y = pred_frame['abs_error']
plt.figure(figsize=(15,4))
plt.plot(x, y, "o", color='red', alpha = 0.1)
plt.legend(loc='best')
plt.show()

In [ ]:
pred_frame['error'] = err_list

In [ ]:
plt.figure(figsize=(15,4))
x = pred_frame['avg_kwh_day_over_invoice']
y = pred_frame['error']
plt.plot(x, y, "o", color='blue', alpha = 0.1)

plt.legend(loc='best')
plt.show()

In [ ]:
x = pred_frame['avg_kwh_day_over_invoice']
y = pred_frame['sq_error']
plt.figure(figsize=(15,4))
plt.plot(x, y, "o", color='yellow', alpha = 0.01)
plt.legend(loc='best')
plt.show()

In [ ]:
plt.figure(figsize=(15,4))
x = pred_frame['avg_kwh_day_over_invoice']

y = pred_frame['sq_error']
plt.plot(x, y, "o", color='yellow', alpha = 0.01)

y = pred_frame['abs_error']
plt.plot(x, y, "o", color='red', alpha = 0.01)
y = pred_frame['error']
plt.plot(x, y, "o", color='blue', alpha = 0.01)

plt.axis([0, 100, -160, 160])
plt.legend(loc='best')
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
x = pred_frame['avg_kwh_day_over_invoice']
sns.distplot(x)

In [ ]:
test.columns

In [ ]:
x = test['tavg_interval_avg']
sns.distplot(x)

In [ ]:
sns.jointplot(x='avg_kwh_day_over_invoice', y='tavg_interval_avg', data=test);

In [ ]:
x = test['avg_kwh_day_over_invoice']
y = test['tavg_interval_avg']
with sns.axes_style("white"):
    sns.jointplot(x=x, y=y, kind="hex", color="k");

In [ ]:
sns.jointplot(x='tavg_interval_avg', y='avg_kwh_day_over_invoice', data=test, kind='kde');